In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from watertap3.truck_pipe_cost_functions import elevation,pipe_costing,truck_costing
import pprint

from pyomo.environ import Var, Expression, NonNegativeReals, Block, ConcreteModel, Constraint, Objective, Param, maximize, SolverFactory,RangeSet
import pyomo.environ as pyo
from idaes.core import FlowsheetBlock
from pyomo.util.infeasible import *
from idaes.core.util.model_statistics import *

In [7]:
bw_df = pd.read_csv('/Users/mhardika/Documents/AMO/GeoToolAll_Methods/Water Source Data/Brackish/brackish_sites_baseline_dwi_5Feb23.csv')

In [58]:
# Combined function for clustered optimization
def clustered_opt_func(ps_state_cluster):
    # For clustered dataframe

    # Create dictionary of clusters
    well_site  = ps_state_cluster['BWSites'].unique()
    well_vol = (ps_state_cluster[ps_state_cluster['BWSites']==well]['BWAvailable'].unique()[0] for well in well_site)
    wells_clustered = dict(zip(well_site,well_vol))
    # print(wells_clustered)
    # Create dictionary of places
    ps_pc = ps_state_cluster['PlaceCode'].unique()
    ps_demand = (ps_state_cluster[ps_state_cluster['PlaceCode']==ps]['PSDemand'].unique()[0] for ps in ps_pc)
    ps_clustered = dict(zip(ps_pc,ps_demand))

    # ps_clustered
    # Sort the dataframe in increasing order of tLCOW
    ps_state_cluster_sorted = ps_state_cluster.sort_values(['tLCOW'],ignore_index=True)
    optimize_network = pd.DataFrame(columns = ps_state_cluster.columns.values)

    for idx,row in ps_state_cluster_sorted.iterrows():
        # Check if demand has already been met and if well is already exhausted
        if wells_clustered[row['BWSites']]>0:
            if ps_clustered[row['PlaceCode']]>0:
                # print(row['PlaceCode'],ps_clustered[row['PlaceCode']], row['BWSites'],wells_clustered[row['BWSites']])
                # Updating the BWAvailable values
                row['BWAvailable'] = wells_clustered[row['BWSites']]
                optimize_network.loc[len(optimize_network)] = row
                # Update demand and well availability
                temp =  ps_clustered[row['PlaceCode']]
                ps_clustered[row['PlaceCode']] = ps_clustered[row['PlaceCode']] -  wells_clustered[row['BWSites']]
                wells_clustered[row['BWSites']] = wells_clustered[row['BWSites']] - temp


    optimize_network1 = optimize_network.sort_values(['PlaceCode','tLCOW']).groupby('PlaceCode')
    optimize_network1 = optimize_network1.apply(pd.DataFrame).set_index('PlaceCode')
    # optimize_network1

    weighted_lcow = []
    cum_ps_demand_frac_met = []
    cum_bw_vol = []

    for ps in optimize_network1.index.unique():
        try:
            # Updated to be supplied volume
            cum_bw_supply = np.array(optimize_network1['TransportVol_m3/s'].loc[ps].cumsum())
            cum_cost = np.array((optimize_network1['TransportVol_m3/s'].loc[ps]*optimize_network1['tLCOW'].loc[ps]).cumsum())
        except AttributeError:
            cum_bw_supply =  np.array([optimize_network1['TransportVol_m3/s'].loc[ps]])
            cum_cost = np.array((optimize_network1['TransportVol_m3/s'].loc[ps]*optimize_network1['tLCOW'].loc[ps]))

        temp = np.divide(cum_cost,cum_bw_supply)
        cum_bw_vol=np.append(cum_bw_vol,cum_bw_supply)
        weighted_lcow = np.append(weighted_lcow, temp,axis=0)
        temp_demand_met = cum_bw_supply/optimize_network1['PSDemand'].loc[ps]
        cum_ps_demand_frac_met = np.append(cum_ps_demand_frac_met,temp_demand_met,axis=0)

    optimize_network1['WeightedtLCOW'] = weighted_lcow
    optimize_network1['Cum_Vol'] = cum_bw_vol
    optimize_network1['Cum_PSDemandFracMet'] = cum_ps_demand_frac_met
    optimize_network1['Cum_PSDemandFracMet'] = np.where(optimize_network1['Cum_PSDemandFracMet']>1,1,optimize_network1['Cum_PSDemandFracMet'])     

    optimize_network_clustered = optimize_network1
    return optimize_network_clustered

In [63]:
# Automating economy of scale calculation
state_codes = ['AL','AZ','AR','CA','CO','CT','FL','GA','ID','IL',
               #'IN','MA','NH','TN','ME',
               'IA','KS','KY','LA','MD','MI','MN','MS','MO',
               'MT','NE','NV','NJ','NM','NY','NC','ND','OH',
               'OK','OR','PA','SC','SD','TX','UT','VA','WA',
               'WV','WI','WY']

# Create lists for results

transport_lcow_clustered_list = []
treatment_lcow_clustered_list = []

for state_alpha in state_codes:
    # Read relevant files
    # PATH = r'\Users\mhardika\Documents\AMO\Analysis Data Files\clustering_results\\' + state_alpha
    # cluster_df = pd.read_csv(PATH + '\\' + state_alpha.lower()+'_cluster.csv').drop(['Unnamed: 0'],axis=1)

    # bw_state = bw_df[bw_df['state_alpha']==state_alpha]

    # Read clustered table
    ps_combined_cluster = pd.read_csv(r'\Users\mhardika\Documents\AMO\Analysis Data Files\Public Supply\state_wise_analysis\\' + state_alpha + '\ps_clustered_v2_'+ state_alpha.lower()+'.csv')
    ps_state_cluster = ps_combined_cluster[ps_combined_cluster['State']==state_alpha]

    # PATH = r'\Users\mhardika\Documents\AMO\Analysis Data Files\clustering_results\\' + state_alpha
    # cluster_bw_df = pd.read_csv(PATH + '\\' + state_alpha.lower()+'_cluster.csv').drop(['Unnamed: 0'],axis=1)

    # Clustered case calculations
    optimize_network_clustered = clustered_opt_func(ps_state_cluster)
    # Save optimized dataframe
    optimize_network_clustered.to_csv(r'\Users\mhardika\Documents\AMO\Analysis Data Files\Public Supply\state_wise_analysis\\' + 
                                      state_alpha + '\ps_clustered_optimized_network_v2_' + state_alpha.lower()+'.csv')

    transport_lcow_clustered = 0
    treatment_lcow_clustered = 0

    vol = 0
    temp = 0 

    for idx,row in optimize_network_clustered.iterrows():
        transport_lcow_clustered = transport_lcow_clustered + row['TransportVol_m3/s']*row['TransportLCOW']
        treatment_lcow_clustered = treatment_lcow_clustered + row['TransportVol_m3/s']*row['LCOW']

        vol = vol + row['TransportVol_m3/s']

    transport_lcow_clustered = transport_lcow_clustered/vol
    treatment_lcow_clustered = treatment_lcow_clustered/vol   

    transport_lcow_clustered_list.append(transport_lcow_clustered)
    treatment_lcow_clustered_list.append(treatment_lcow_clustered)


c:\Users\mhardika\miniconda3\envs\watertap3\lib\site-packages\pandas\core\indexes\base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
c:\Users\mhardika\miniconda3\envs\watertap3\lib\site-packages\pandas\core\indexes\base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
c:\Users\mhardika\miniconda3\envs\watertap3\lib\site-packages\pandas\core\indexes\base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
c:\Users\mhardika\miniconda3\envs\watertap3\lib\site-packages\pandas\core\indexes\base.py:6999: FutureWarning: In a future version, the Ind

In [65]:
results_df = pd.DataFrame(columns=['state_alpha',#'transport_lcow_unclustered','treatment_lcow_unclustered',
                                   'transport_lcow_clustered','treatment_lcow_clustered'])
results_df['state_alpha'] = state_codes
# results_df['transport_lcow_unclustered'] = transport_lcow_unclustered_list
# results_df['treatment_lcow_unclustered'] = treatment_lcow_unclustered_list
results_df['transport_lcow_clustered'] = transport_lcow_clustered_list
results_df['treatment_lcow_clustered'] = treatment_lcow_clustered_list

results_df.head(5)
results_df.to_csv(r'C:\Users\mhardika\Documents\AMO\Analysis Data Files\Public Supply\ps_rough_economy_of_scale_output_v2.csv')